## CH1: Overloading and Multiple Inheritance

##### Overloading the + operator

In [14]:
class Team:
    def __init__(self, team_members):
        self.team_members = team_members
    
    def __add__(self, other):
        if type(other) == Employee:
            return Team(self.team_members + [other.name])
        else:
            return Team(self.team_members + other.team_members)
    
class Employee:
    def __init__(self, name, title):
        self.name, self.title = name, title

    def __add__(self, other):
        return Team([self.name, other.name])

In [15]:
emp1 = Employee("Alice", "engineer")
emp2 = Employee("Bob", "musician")
emp3 = Employee("Joe", "manager")
emp4 = Employee("Anna", "engineer")

In [16]:
team1 = emp1 + emp2
team2 = emp3 + emp4

print(team1.team_members)
print(team2.team_members)

team3 = team1 + team2
print(team3.team_members)

team4 = team3 + Employee("Kat", "engineer")
print(team4.team_members)

['Alice', 'Bob']
['Joe', 'Anna']
['Alice', 'Bob', 'Joe', 'Anna']
['Alice', 'Bob', 'Joe', 'Anna', 'Kat']


##### Method Resolution Order (MRO)

In [22]:
class Engineer:
    def __init__(self, name, company):
        self.name, self.company = name, company
    
    def introduce(self):
        print(f"I am an engineer {self.name} who works at {self.company}.")

class Student:
    def __init__(self, name, university):
        self.name, self.university = name, university

    def introduce(self):
        print(f"I am {self.name}. I am a student at {self.university}.")

class Intern(Student, Engineer):
    def __init__(self, name, university, company):
        Student.__init__(self, name, university)
        Engineer.__init__(self, name, company)

In [23]:
omer = Intern("Omer", "METU", "PyTech")
omer.introduce()
print(Intern.__mro__)

I am Omer. I am a student at METU.
(<class '__main__.Intern'>, <class '__main__.Student'>, <class '__main__.Engineer'>, <class 'object'>)


## CH2: Custom Class Features and Type Hints

##### Type hints

In [25]:
from typing import List

class Student:
    def __init__(self, name: str, id: int, courses: List[str]) -> None:
        self.name: str = name
        self.id: int = id
        self.courses: List[str] = courses
    
    def print_courses(self) -> None:
        for course in self.courses:
            print(course.upper())

omer = Student("Omer", 1, ["python", "java", "dart"])
omer.print_courses()
        

PYTHON
JAVA
DART


##### Descriptors

In [27]:
class Student:
    def __init__(self, name, ssn):
        self.name, self.ssn = name, ssn
    
    @property
    def ssn(self):
        return "XXX-XX-" + self._ssn[-4:]
    
    @ssn.setter
    def ssn(self, new_ssn):
        if len(new_ssn) == 11:
            self._ssn = new_ssn
    
    @ssn.deleter
    def ssn(self):
        print("Can't delete SSN")

In [28]:
omer = Student("Omer", "123-45-6789")
print(omer.ssn)

XXX-XX-6789


In [34]:
omer.ssn = "23456"
print(omer.ssn)

omer.ssn = "821-11-9029"
print(omer.ssn)

del omer.ssn
print(omer.ssn)

XXX-XX-9029
XXX-XX-9029
Can't delete SSN
XXX-XX-9029


##### Attribute Access

In [57]:
class Student:
    def __init__(self, ssn):
        self.ssn = ssn

    def __getattr__(self, name):
        self.__setattr__(name, None)

    def __setattr__(self, name, value):
        if value is None:
            print(f"Setting placeholder for {name}")

        elif (isinstance(value, str)) and (name != "ssn"):
            print(f"Setting {name} = {value}")
            self.__dict__[name] = value
        
        self.__dict__[name] = value

In [58]:
omer = Student("1234")
omer.major = "Ceng"
print(omer.major)
print(omer.school)
print(omer.__dict__)

Setting major = Ceng
Ceng
Setting placeholder for school
None
{'ssn': '1234', 'major': 'Ceng', 'school': None}


##### Custom iterators

In [61]:
import random


class CoinFlips:
    def __init__(self, number_of_flips):
        self.number_of_flips = number_of_flips
        self.counter = 0
    
    def __iter__(self):
        return self  # Return a reference of the iterator
    
    # Flip the next coin, return the output
    def __next__(self):
        if self.counter < self.number_of_flips:
            self.counter += 1
            return random.choice(["H", "T"])
        else:
            raise StopIteration

In [68]:
three_flips = CoinFlips(3)
for flip in three_flips:
    print(flip)

T
H
T


In [71]:
three_flips = CoinFlips(3)

while True:
    try:
        print(next(three_flips))
    except StopIteration:
        print("Completed all flips!")
        break

T
H
T
Completed all flips!


## CH3: Object-oriented design patterns

##### Abstract base classes

In [72]:
from abc import ABC, abstractmethod

class School(ABC):
    @abstractmethod
    def enroll(self):
        pass

    def graduate(self):
        print("Congrats on graduating!")

class University(School):
    def enroll(self):
        print("Welcome to university!")

In [74]:
uni = University()
uni.enroll()
uni.graduate()

Welcome to university!
Congrats on graduating!


##### Interfaces

In [76]:
from abc import ABC, abstractmethod

class Course(ABC):
    @abstractmethod
    def assign_homework(self, assignment_number, due_date):
        pass

    @abstractmethod
    def grade_assignment(self, assignment_number):
        pass

class ProgrammingCourse(Course):
    def __init__(self, course_name):
        self.course_name = course_name
    
    def assign_homework(self, assignment_number, due_date):
        print(f"Homework {assignment_number} assigned! Due date: {due_date}")
    
    def grade_assignment(self, assignment_number):
        print(f"Homework {assignment_number} graded!")

p_course = ProgrammingCourse("Python")
p_course.assign_homework(100, "7 June")
p_course.grade_assignment(100)

Homework 100 assigned! Due date: 7 June
Homework 100 graded!


##### Factory Methods

In [77]:
class Resource(ABC):
    @abstractmethod
    def reference(self, topic):
        pass

class Textbook(Resource):
    def __init__(self):
        self.index = {"OOP": ["Inheritance"]}
    def reference(self, topic):
        print(f"Referencing {topic} using a textbook")
        return self.index.get(topic)
    
class Video(Resource):
    def __init__(self):
        self.index = {"OOP": ["Multiple Inheritance"]}
    def reference(self, topic):
        print(f"Referencing {topic} using a video")
        return self.index.get(topic)

In [80]:
class Student:
    # Factory method to return Resource
    def _get_resource(self, resource_type):
        if resource_type == "Textbook":
            return Textbook()
    
        elif resource_type == "Video":
            return Video()

    def explore_topic(self, resource_type, topic):
        resource = self._get_resource(resource_type) # Retrieve the resource
        print(resource.reference(topic))

In [81]:
# Create a Student object, then have it reference a textbook
lester = Student()
lester.explore_topic("Textbook", "OOP")

# Each to switch to another resource
lester.explore_topic("Video", "OOP")

Referencing OOP using a textbook
['Inheritance']
Referencing OOP using a video
['Multiple Inheritance']
